In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import OPTICS
import json
import time
boundingbox = []
for i in range(10):
    time_start = time.time()  # time start
    file_data = np.fromfile('../me5413_homework1/me5413/1_lidar/lidar_data/frame'+str(i+1)+'.pcd.bin', dtype=np.float32)
    #reshape the data
    points = file_data.reshape((-1, 5))[:, :4]
    #Optics
    cluster = OPTICS(max_eps=np.inf,min_samples=45, cluster_method='dbscan',eps=7.5).fit(points)
    labels=cluster.labels_
    print(labels)
    #How many clusters and noise
    n_clusters=len(set(labels))-(1 if -1 in labels else 0)
    n_noise=list(labels).count(-1)
    print("Estimated number of clusters:%d"% n_clusters)
    print("Estimated number of noise points: %d"% n_noise)
    #distribute color to label
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
    #create figure
    fig=plt.figure(dpi=130)
    ax=fig.add_subplot(111,projection='3d')
    plt.title('Points Cloud clustered by OPTICS')
    xc=np.zeros(len(unique_labels))
    yc=np.zeros(len(unique_labels))
    zc=np.zeros(len(unique_labels))
    width=np.zeros(len(unique_labels))
    height=np.zeros(len(unique_labels))
    deep=np.zeros(len(unique_labels))
    for k, col in zip(unique_labels, colors):
        if k != -1:
            class_member = (labels == k)  # not discrete
            xyz = points[class_member]  # choose all these points
            xmax = np.max(xyz[:, 0])
            xmin = np.min(xyz[:, 0])
            ymax = np.max(xyz[:, 1])
            ymin = np.min(xyz[:, 1])
            zmax = np.max(xyz[:, 2])
            zmin = np.min(xyz[:, 2])
            x = [xmin, xmax, xmax, xmin, xmin, xmax, xmax, xmin]
            y = [ymax, ymax, ymax, ymax, ymin, ymin, ymin, ymin]
            z = [zmax, zmax, zmin, zmin, zmax, zmax, zmin, zmin]
            A, B, C, D, E, F, G, H = zip(x, y, z)
            # draw boundingbox
            lines_1 = zip(A, B, C, D, A, E, F, G, C, B, F,G,H,E,F,G,H,D)
            ax.plot3D(*lines_1, c=tuple(col), marker='.', ms=1,alpha=0.7)
            width[k]=xmax-xmin
            height[k]=zmax-zmin
            deep[k]=ymax-ymin
            xc[k]=width[k]*0.5
            yc[k]= deep[k]*0.5
            zc[k]= height[k]*0.5
            # draw piontcloud
            ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2], s=xyz[:,3]*0.3,c=tuple(col), marker='.')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    time_end = time.time()  # time end
    time_c = time_end - time_start  # time used
    print('time cost', time_c, 's')
    # plt.show()
    plt.savefig('result_pic/OPTICS/frame' + str(i + 1) + '_OPTICS.png')
    # save as json
    b = {}
    b["frame"] = "frame" + str(i + 1)
    b["translation"] = xc.tolist(), yc.tolist(), zc.tolist()
    b["size"] = width.tolist(), deep.tolist(), height.tolist()
    boundingbox.append(b)
filename='OPTICS.json'
with open(filename,'w') as file_obj:
        json.dump(boundingbox,file_obj)